# PySpark 3.5 → DataHub (acryl-spark)

[![PySpark](https://img.shields.io/badge/PySpark-3.5-262A38?style=flat-square&logo=apachespark&logoColor=E36B22&labelColor=262A38)](https://spark.apache.org/docs/3.5.7/api/python/user_guide/index.html)
[![Scala](https://img.shields.io/badge/Scala-2.12-262A38?style=flat-square&logo=scala&logoColor=E03E3C&labelColor=262A38)](https://sdkman.io/)
[![JDK](https://img.shields.io/badge/JDK-17-35667C?style=flat&logo=openjdk&logoColor=FFFFFF&labelColor=1D213B)](https://sdkman.io/)
[![Acryl-Spark](https://img.shields.io/badge/acryl--spark--lineage-262A38?style=flat-square&logo=lineageos&logoColor=73A4BC&labelColor=262A38)](https://docs.datahub.com/docs/metadata-integration/java/acryl-spark-lineage)

Read from the staging HackerNews RSS tables (`stg_hot_articles`, `stg_newest_articles`) and write
the combined result into the `hackernews_rss` dataset on BigQuery.

**Notes:** 
This notebook requires **Spark 3.5** as it uses dependencies bound to the Spark version and/or the Scala version (2.12) that is bundled with

### SparkSession Setup

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark_jars = ",".join([
    "com.google.cloud.spark:spark-3.5-bigquery:0.44.0",
    "io.acryl:acryl-spark-lineage_2.12:0.2.18",
])

In [3]:
spark = (
    SparkSession.builder
    .appName("jupyter-acryl-datahub")
    .master("local[*]")
    .config("spark.jars.packages", spark_jars)
    .config("spark.extraListeners", "datahub.spark.DatahubSparkListener")
    .config("spark.datahub.rest.server", "http://localhost:9090")
    .config("spark.datahub.metadata.dataset.materialize", "true")
    .config("spark.datahub.capture_spark_plan", "true")
    .getOrCreate()
)

26/02/13 07:34:46 WARN Utils: Your hostname, Brunos-M1-Max-MBP-16.local resolves to a loopback address: 127.0.0.1; using 192.168.15.5 instead (on interface en0)
26/02/13 07:34:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/iobruno/.ivy2/cache
The jars for the packages stored in: /Users/iobruno/.ivy2/jars
com.google.cloud.spark#spark-3.5-bigquery added as a dependency
io.acryl#acryl-spark-lineage_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f0e41c16-3bbf-4d4f-8061-4c546bcd5682;1.0
	confs: [default]
	found com.google.cloud.spark#spark-3.5-bigquery;0.44.0 in central


:: loading settings :: url = jar:file:/Users/iobruno/Vault/data-catalog-labs/spark/.venv/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.google.cloud.spark#spark-bigquery-dsv2-common;0.44.0 in central
	found com.google.cloud.spark#spark-bigquery-connector-common;0.44.0 in central
	found com.google.cloud.spark#bigquery-connector-common;0.44.0 in central
	found com.google.api.grpc#grpc-google-cloud-bigquerystorage-v1;3.16.1 in central
	found io.grpc#grpc-api;1.74.0 in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found com.google.errorprone#error_prone_annotations;2.41.0 in central
	found io.grpc#grpc-stub;1.74.0 in central
	found org.codehaus.mojo#animal-sniffer-annotations;1.24 in central
	found io.grpc#grpc-protobuf;1.74.0 in central
	found com.google.api.grpc#proto-google-common-protos;2.59.2 in central
	found com.google.protobuf#protobuf-java;3.25.8 in central
	found com.google.api.grpc#proto-google-cloud-bigquerystorage-v1;3.16.1 in central
	found com.google.api#api-common;2.51.2 in central
	found com.google.auto.value#auto-value-annotations;1.11.0 in central
	found javax.annotation#jav

### Read Staging Tables

In [4]:
GCP_PROJECT = "iobruno-gcp-labs"
STG_DATASET = "stg_hackernews_rss"

df_hot = (
    spark.read
    .format("bigquery")
    .option("table", f"{GCP_PROJECT}.{STG_DATASET}.stg_hot_articles")
    .option("viewsEnabled", "true")
    .option("materializationDataset", STG_DATASET)
    .load()
)

df_hot.printSchema()
df_hot.show(5, truncate=False)

26/02/13 07:34:51 WARN SparkOpenLineageExtensionVisitorWrapper: Different classloaders detected for openlineage-spark integration and Spark connector. This may cause extension loading issues. For optimal compatibility, ensure both libraries are loaded using the same classloader by: 
1. Placing both libraries in the /usr/lib/spark/jars directory, or 
2. Loading both libraries through the --jars parameter.


root
 |-- uid: string (nullable = true)
 |-- title: string (nullable = true)
 |-- username: string (nullable = true)
 |-- url: string (nullable = true)
 |-- redirect_url: string (nullable = true)
 |-- published_at: timestamp (nullable = true)



26/02/13 07:34:58 WARN SparkOpenLineageExtensionVisitorWrapper: Different classloaders detected for openlineage-spark integration and Spark connector. This may cause extension loading issues. For optimal compatibility, ensure both libraries are loaded using the same classloader by: 
1. Placing both libraries in the /usr/lib/spark/jars directory, or 
2. Loading both libraries through the --jars parameter.
26/02/13 07:34:58 WARN DatasetVersionDatasetFacetUtils: Couldn't find identifier for dataset in plan RelationV2[uid#0, title#1, username#2, url#3, redirect_url#4, published_at#5]  stg_hot_articles

ANTLR Tool version 4.5 used for code generation does not match the current runtime version 4.9.3
ANTLR Runtime version 4.5 used for parser compilation does not match the current runtime version 4.9.3
ANTLR Tool version 4.5 used for code generation does not match the current runtime version 4.9.3
ANTLR Runtime version 4.5 used for parser compilation does not match the current runtime version 

+--------------------------------+--------------------------------------------------------------------------------+------------+---------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+-------------------+
|uid                             |title                                                                           |username    |url                                          |redirect_url                                                                                                           |published_at       |
+--------------------------------+--------------------------------------------------------------------------------+------------+---------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+-------------------+
|7b54e42263906dd156b7743931038f1e|Byte magazine artist 

In [5]:
df_newest = (
    spark.read
    .format("bigquery")
    .option("table", f"{GCP_PROJECT}.{STG_DATASET}.stg_newest_articles")
    .option("viewsEnabled", "true")
    .option("materializationDataset", STG_DATASET)
    .load()
)

df_newest.printSchema()
df_newest.show(5, truncate=False)

root
 |-- uid: string (nullable = true)
 |-- title: string (nullable = true)
 |-- username: string (nullable = true)
 |-- url: string (nullable = true)
 |-- redirect_url: string (nullable = true)
 |-- published_at: timestamp (nullable = true)



26/02/13 07:35:05 WARN SparkOpenLineageExtensionVisitorWrapper: Different classloaders detected for openlineage-spark integration and Spark connector. This may cause extension loading issues. For optimal compatibility, ensure both libraries are loaded using the same classloader by: 
1. Placing both libraries in the /usr/lib/spark/jars directory, or 
2. Loading both libraries through the --jars parameter.
26/02/13 07:35:05 WARN DatasetVersionDatasetFacetUtils: Couldn't find identifier for dataset in plan RelationV2[uid#43, title#44, username#45, url#46, redirect_url#47, published_at#48]  stg_newest_articles



+--------------------------------+-------------------------------------------------------------------------------+------------+---------------------------------------------+---------------------------------------------------------------------------------------------------------+-------------------+
|uid                             |title                                                                          |username    |url                                          |redirect_url                                                                                             |published_at       |
+--------------------------------+-------------------------------------------------------------------------------+------------+---------------------------------------------+---------------------------------------------------------------------------------------------------------+-------------------+
|2b177f05631c0419d3df8e6fe106512c|Claude prefers JSON over Markdown                                 

### Combine Articles

In [6]:
from pyspark.sql.functions import lit

df_hot_tagged = df_hot.withColumn("source_feed", lit("hot"))
df_newest_tagged = df_newest.withColumn("source_feed", lit("newest"))

df_articles = df_hot_tagged.unionByName(df_newest_tagged)

print(f"Total articles: {df_articles.count()}")
df_articles.show(10, truncate=False)

26/02/13 07:35:09 WARN SparkOpenLineageExtensionVisitorWrapper: Different classloaders detected for openlineage-spark integration and Spark connector. This may cause extension loading issues. For optimal compatibility, ensure both libraries are loaded using the same classloader by: 
1. Placing both libraries in the /usr/lib/spark/jars directory, or 
2. Loading both libraries through the --jars parameter.


Total articles: 40


26/02/13 07:35:10 WARN SparkOpenLineageExtensionVisitorWrapper: Different classloaders detected for openlineage-spark integration and Spark connector. This may cause extension loading issues. For optimal compatibility, ensure both libraries are loaded using the same classloader by: 
1. Placing both libraries in the /usr/lib/spark/jars directory, or 
2. Loading both libraries through the --jars parameter.
26/02/13 07:35:10 WARN DatasetVersionDatasetFacetUtils: Couldn't find identifier for dataset in plan RelationV2[uid#0, title#1, username#2, url#3, redirect_url#4, published_at#5]  stg_hot_articles

26/02/13 07:35:10 WARN DatasetVersionDatasetFacetUtils: Couldn't find identifier for dataset in plan RelationV2[uid#43, title#44, username#45, url#46, redirect_url#47, published_at#48]  stg_newest_articles



+--------------------------------+--------------------------------------------------------------------------------+------------+---------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+-------------------+-----------+
|uid                             |title                                                                           |username    |url                                          |redirect_url                                                                                                           |published_at       |source_feed|
+--------------------------------+--------------------------------------------------------------------------------+------------+---------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+-------------------+-----------+
|7b54e42263906dd156

### Write to BigQuery

In [7]:
TARGET_DATASET = "hackernews_rss"
TARGET_TABLE = f"{GCP_PROJECT}.{TARGET_DATASET}.articles_spark"

(
    df_articles.write
    .format("bigquery")
    .option("table", TARGET_TABLE)
    .option("writeMethod", "direct")
    .mode("overwrite")
    .save()
)

print(f"Written to {TARGET_TABLE}")

26/02/13 07:35:11 WARN SparkOpenLineageExtensionVisitorWrapper: Different classloaders detected for openlineage-spark integration and Spark connector. This may cause extension loading issues. For optimal compatibility, ensure both libraries are loaded using the same classloader by: 
1. Placing both libraries in the /usr/lib/spark/jars directory, or 
2. Loading both libraries through the --jars parameter.
26/02/13 07:35:11 WARN DatasetVersionDatasetFacetUtils: Couldn't find identifier for dataset in plan RelationV2[uid#0, title#1, username#2, url#3, redirect_url#4, published_at#5]  stg_hot_articles

26/02/13 07:35:11 WARN DatasetVersionDatasetFacetUtils: Couldn't find identifier for dataset in plan RelationV2[uid#43, title#44, username#45, url#46, redirect_url#47, published_at#48]  stg_newest_articles

26/02/13 07:35:11 WARN ColumnLevelLineageUtils: DEPRECATION WARNING: The columnLineage.datasetLineageEnabled configuration is set to false. This flag will default to true in the future ve

Written to iobruno-gcp-labs.hackernews_rss.articles_spark


26/02/13 07:35:19 WARN ColumnLevelLineageUtils: DEPRECATION WARNING: The columnLineage.datasetLineageEnabled configuration is set to false. This flag will default to true in the future versions. To avoid this warning, explicitly set it to true. This warning will automatically be removed once the default is switched to true.
